# We Read The Data from SQLite file
#### Data is downloaded from kaggle <a href="https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews?resource=download">Link</a>

In [1]:
import sqlite3 as sql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import string

In [8]:
connection=sql.connect('../Data Sets/database.sqlite') #This line opens a connection to my DB

In [9]:
print(data)

In [33]:
pd_data=pd.read_sql_query("""
select * 
from Reviews
where Score != 3
""",connection) #Here we get the only positive or negative data

In [34]:
print(pd_data.head())

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

In [35]:
pd_data.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [36]:
def sep(x):
    if x<3:
        return 'Negative'
    else :
        return "Positive"

In [37]:
actual_score=pd_data['Score']

In [38]:
print(len(actual_score))

525814


In [29]:
pd_data.Score.isnull().sum()

0

In [39]:
#as there is not any NAN value exist in pd_data so we use .apply in place of .map by considering of getting a pandas data frame
scoreCard=pd_data.Score
scoreCard=scoreCard.map(sep)

In [40]:
scoreCard

0         Positive
1         Negative
2         Positive
3         Negative
4         Positive
            ...   
525809    Positive
525810    Negative
525811    Positive
525812    Positive
525813    Positive
Name: Score, Length: 525814, dtype: object

In [41]:
pd_data.Score=scoreCard

In [42]:
pd_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,Negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


# Data Cleaning: Data Duplication
\